In [62]:
import pandas as pd
import datetime
from tqdm import tqdm
import math
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
import numpy as np
import time

In [63]:
data = pd.read_csv('data/external/data',sep="\t",header=None,names=["last_changed","entity_id","state","activity","log"])
data[~data['activity'].isna()]

/var/folders/5p/f0zx4qd55tqd2tj2yx17fq9h0000gn/T/ipykernel_2401/1845251821.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/external/data',sep="\t",header=None,names=["last_changed","entity_id","state","activity","log"])


,last_changed,entity_id,state,activity,log
86,2009-10-16 03:55:53.000080,M021,ON,Bed_to_Toilet begin,NaN
102,2009-10-16 03:58:28.000002,M025,ON,Bed_to_Toilet end,NaN
105,2009-10-16 03:58:44.000068,M020,ON,Sleep begin,NaN
328,2009-10-16 08:40:01.000075,M021,ON,Sleep end,NaN
384,2009-10-16 08:42:01.000077,M022,ON,Morning_Meds begin,NaN
...,...,...,...,...,...
433241,2010-01-06 12:26:09.000035,M002,ON,Leave_Home end,NaN
433275,2010-01-06 12:26:57.000079,M012,ON,Kitchen_Activity begin,NaN
433396,2010-01-06 12:34:19.000047,M017,ON,Master_Bathroom begin,NaN
433449,2010-01-06 12:39:40.000080,M017,ON,Master_Bathroom end,NaN


In [64]:
data['last_changed'] = pd.to_datetime(data['last_changed'], format='%Y-%m-%d %H:%M:%S').dt.floor('S')

In [65]:
#Feature engineering
data = data[data['last_changed'].notna()]
data['timestamp'] = pd.to_datetime(data['last_changed']).dt.floor('T')
data['id_state'] = data['entity_id'] + '-' + data['state']
startDate =  pd.to_datetime('2009-10-16 00:00:00')
endDate = startDate + datetime.timedelta(days=2)
data = data.loc[(data['last_changed'] >= startDate)
                     & (data['last_changed'] <= endDate)]
data = data[data['entity_id'].str.contains('T')==False]
data.iloc[75:85]

,last_changed,entity_id,state,activity,log,timestamp,id_state
79,2009-10-16 03:55:27,M021,OFF,NaN,NaN,2009-10-16 03:55:00,M021-OFF
80,2009-10-16 03:55:35,M021,ON,NaN,NaN,2009-10-16 03:55:00,M021-ON
81,2009-10-16 03:55:39,M021,OFF,NaN,NaN,2009-10-16 03:55:00,M021-OFF
82,2009-10-16 03:55:43,M021,ON,NaN,NaN,2009-10-16 03:55:00,M021-ON
83,2009-10-16 03:55:46,M021,OFF,NaN,NaN,2009-10-16 03:55:00,M021-OFF
84,2009-10-16 03:55:48,M021,ON,NaN,NaN,2009-10-16 03:55:00,M021-ON
85,2009-10-16 03:55:50,M021,OFF,NaN,NaN,2009-10-16 03:55:00,M021-OFF
86,2009-10-16 03:55:53,M021,ON,Bed_to_Toilet begin,NaN,2009-10-16 03:55:00,M021-ON
87,2009-10-16 03:55:58,M021,OFF,NaN,NaN,2009-10-16 03:55:00,M021-OFF
88,2009-10-16 03:56:05,M028,ON,NaN,NaN,2009-10-16 03:56:00,M028-ON


In [66]:
activities = data[~data['activity'].isna()]
activities['id_state'] = activities['activity']
activities

/var/folders/5p/f0zx4qd55tqd2tj2yx17fq9h0000gn/T/ipykernel_2401/3315814589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities['id_state'] = activities['activity']


,last_changed,entity_id,state,activity,log,timestamp,id_state
86,2009-10-16 03:55:53,M021,ON,Bed_to_Toilet begin,NaN,2009-10-16 03:55:00,Bed_to_Toilet begin
102,2009-10-16 03:58:28,M025,ON,Bed_to_Toilet end,NaN,2009-10-16 03:58:00,Bed_to_Toilet end
105,2009-10-16 03:58:44,M020,ON,Sleep begin,NaN,2009-10-16 03:58:00,Sleep begin
328,2009-10-16 08:40:01,M021,ON,Sleep end,NaN,2009-10-16 08:40:00,Sleep end
384,2009-10-16 08:42:01,M022,ON,Morning_Meds begin,NaN,2009-10-16 08:42:00,Morning_Meds begin
...,...,...,...,...,...,...,...
10292,2009-10-17 23:16:16,M021,ON,Sleep begin,NaN,2009-10-17 23:16:00,Sleep begin
10347,2009-10-17 23:30:25,M028,OFF,Sleep end,NaN,2009-10-17 23:30:00,Sleep end
10349,2009-10-17 23:30:29,M025,ON,Bed_to_Toilet begin,NaN,2009-10-17 23:30:00,Bed_to_Toilet begin
10370,2009-10-17 23:33:43,M025,OFF,Bed_to_Toilet end,NaN,2009-10-17 23:33:00,Bed_to_Toilet end


In [67]:
data = data.append(activities, ignore_index=True)
data = data.sort_values(by=['last_changed'])
data = data.drop(columns=['entity_id', 'state', 'activity', 'log'])
data['last_changed'] = pd.to_datetime(data['last_changed'])
startDate = pd.to_datetime('2009-10-16 00:00:00') #ADD STARTDATE FROM WHERE TO BEGIN WEEK
endDate = startDate + datetime.timedelta(days=14)
data = data.loc[(data['last_changed'] >= startDate)
                     & (data['last_changed'] <= endDate)]

df = data
df

/var/folders/5p/f0zx4qd55tqd2tj2yx17fq9h0000gn/T/ipykernel_2401/3421287958.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(activities, ignore_index=True)


,last_changed,timestamp,id_state
0,2009-10-16 00:01:04,2009-10-16 00:01:00,M017-ON
1,2009-10-16 00:01:06,2009-10-16 00:01:00,M009-ON
2,2009-10-16 00:01:07,2009-10-16 00:01:00,M017-OFF
3,2009-10-16 00:01:08,2009-10-16 00:01:00,M019-ON
4,2009-10-16 00:01:09,2009-10-16 00:01:00,M009-OFF
...,...,...,...
10331,2009-10-17 23:42:44,2009-10-17 23:42:00,M010-OFF
10332,2009-10-17 23:42:44,2009-10-17 23:42:00,M009-ON
10333,2009-10-17 23:42:46,2009-10-17 23:42:00,M019-ON
10334,2009-10-17 23:42:47,2009-10-17 23:42:00,M009-OFF


In [68]:
df[~df['id_state'].str.contains("D")][~df['id_state'].str.contains("M")]

/var/folders/5p/f0zx4qd55tqd2tj2yx17fq9h0000gn/T/ipykernel_2401/3146898642.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[~df['id_state'].str.contains("D")][~df['id_state'].str.contains("M")]


,last_changed,timestamp,id_state
10336,2009-10-16 03:55:53,2009-10-16 03:55:00,Bed_to_Toilet begin
10337,2009-10-16 03:58:28,2009-10-16 03:58:00,Bed_to_Toilet end
10338,2009-10-16 03:58:44,2009-10-16 03:58:00,Sleep begin
10339,2009-10-16 08:40:01,2009-10-16 08:40:00,Sleep end
10342,2009-10-16 08:43:59,2009-10-16 08:43:00,Watch_TV begin
...,...,...,...
10455,2009-10-17 23:16:16,2009-10-17 23:16:00,Sleep begin
10456,2009-10-17 23:30:25,2009-10-17 23:30:00,Sleep end
10457,2009-10-17 23:30:29,2009-10-17 23:30:00,Bed_to_Toilet begin
10459,2009-10-17 23:33:43,2009-10-17 23:33:00,Sleep begin


In [69]:
interval = 5

In [70]:
#Dynamically add start and end dates to bins based on first and last value
startDate = df['last_changed'].head(1).item().floor('T')
endDate = df['last_changed'].tail(1).item().floor('T')
binLength = interval
bins = pd.date_range(start=startDate, end=endDate, freq=f'{binLength}S')
print(f'Bins created with start date {startDate} and end date {endDate}')

Bins created with start date 2009-10-16 00:01:00 and end date 2009-10-17 23:42:00


In [71]:
binLists = []
iteration = 0
for bin in tqdm(bins, desc="Generating bins"):
    startStop = []
    startStop.append(bins[iteration])
    startStop.append(bins[iteration + 1])
    binLists.append(startStop)
    iteration += 1
    if iteration == len(bins) - 1:
        break

Generating bins:  29%|██▉       | 10001/34333 [00:00<00:00, 48608.12it/s]

Generating bins: 100%|█████████▉| 34331/34333 [00:00<00:00, 50885.03it/s]


In [72]:
#Fill all bins with state changes in that timeslot
startTime = time.time()
timeslotsItemsets = []
for binList in binLists:
     current = df[(df.timestamp >= binList[0]) & (df.timestamp <= binList[0])]['id_state']
     if len(current) > 0:
          lastIteration = current
          timeslotsItemsets.append(list(current))
endTime = time.time()
runningTime = endTime - startTime
print(f'Generated {len(timeslotsItemsets)} itemsets in {runningTime} seconds!')

Generated 970 itemsets in 19.731499910354614 seconds!


In [73]:
def transaction_encoder(data):
    te = TransactionEncoder()
    te_ary = te.fit(data).transform(data)
    return pd.DataFrame(te_ary, columns=te.columns_)

In [74]:
encoded = transaction_encoder(timeslotsItemsets)

In [81]:
frequent = fpgrowth(encoded, min_support=0.01, use_colnames=True)
frequent

,support,itemsets
0,0.096907,(M019-ON)
1,0.096907,(M019-OFF)
2,0.077320,(M009-ON)
3,0.076289,(M009-OFF)
4,0.059794,(M017-OFF)
...,...,...
60154,0.010309,"(M023-OFF, M014-OFF, M014-ON, M015-ON, M015-OF..."
60155,0.010309,"(M023-OFF, M014-OFF, M015-ON, M015-OFF, D003-C..."
60156,0.010309,"(M014-OFF, M014-ON, M015-ON, M015-OFF, D003-CL..."
60157,0.010309,"(M023-OFF, M014-OFF, M014-ON, M015-OFF, D003-C..."


In [82]:
rules = association_rules(frequent, metric="confidence", min_threshold=0.0)
rules["consequents"] = rules["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
rules["antecedents"] = rules["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

KeyboardInterrupt: 

In [ ]:
rules[~rules['consequents'].str.contains("M")]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [79]:
antecedents_include = ['M', 'D']
consequents_exclude = ['M', 'D']
filteredRules = rules[(rules['antecedents'].str.contains('|'.join(antecedents_include))) & ~(rules['consequents'].str.contains('|'.join(consequents_exclude)))]
filteredRules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [80]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,M019-ON,M015-OFF,0.096907,0.135052,0.020619,0.212766,1.575443,0.007531,1.098718
1,M015-OFF,M019-ON,0.135052,0.096907,0.020619,0.152672,1.575443,0.007531,1.065812
2,M008-OFF,M019-ON,0.347423,0.096907,0.049485,0.142433,1.469790,0.015817,1.053087
3,M019-ON,M008-OFF,0.096907,0.347423,0.049485,0.510638,1.469790,0.015817,1.333528
4,M019-ON,M008-ON,0.096907,0.348454,0.048454,0.500000,1.434911,0.014686,1.303093
...,...,...,...,...,...,...,...,...,...
425643,"M001-ON, M002-OFF","M001-OFF, M002-ON",0.027835,0.024742,0.024742,0.888889,35.925926,0.024054,8.777320
425644,M001-OFF,"M002-ON, M001-ON, M002-OFF",0.034021,0.027835,0.024742,0.727273,26.127946,0.023795,3.564605
425645,M002-ON,"M001-OFF, M001-ON, M002-OFF",0.032990,0.024742,0.024742,0.750000,30.312500,0.023926,3.901031
425646,M001-ON,"M001-OFF, M002-ON, M002-OFF",0.034021,0.024742,0.024742,0.727273,29.393939,0.023901,3.575945
